## Installing Dependencies

In [1]:
pip install surprise

  Using cached surprise-0.1-py2.py3-none-any.whl (1.8 kB)
  Using cached scikit-surprise-1.1.1.tar.gz (11.8 MB)
  Running setup.py clean for scikit-surprise
Failed to build scikit-surprise
    Running setup.py install for scikit-surprise: startedNote: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'F:\ANUBHUTI\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\ANUBHUTI\\AppData\\Local\\Temp\\pip-install-w_f7vsk8\\scikit-surprise_cf65bcdec9654cdba7cc20802f31d47d\\setup.py'"'"'; __file__='"'"'C:\\Users\\ANUBHUTI\\AppData\\Local\\Temp\\pip-install-w_f7vsk8\\scikit-surprise_cf65bcdec9654cdba7cc20802f31d47d\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\ANUBHUTI\AppData\Local\Temp\pip-wheel-7uu93gru'
       cwd: C:\Users\ANUBHUTI\AppData\Local\Temp\pip-install-w_f7vsk8\scikit-surprise_cf65bcdec9654cdba7cc20802f31d47d\
  Complete output (49 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\surprise
  copying surprise\accuracy.py -> 


    Running setup.py install for scikit-surprise: finished with status 'error'



  copying surprise\model_selection\validation.py -> build\lib.win-amd64-3.8\surprise\model_selection
  copying surprise\model_selection\__init__.py -> build\lib.win-amd64-3.8\surprise\model_selection
  creating build\lib.win-amd64-3.8\surprise\prediction_algorithms
  copying surprise\prediction_algorithms\algo_base.py -> build\lib.win-amd64-3.8\surprise\prediction_algorithms
  copying surprise\prediction_algorithms\baseline_only.py -> build\lib.win-amd64-3.8\surprise\prediction_algorithms
  copying surprise\prediction_algorithms\knns.py -> build\lib.win-amd64-3.8\surprise\prediction_algorithms
  copying surprise\prediction_algorithms\predictions.py -> build\lib.win-amd64-3.8\surprise\prediction_algorithms
  copying surprise\prediction_algorithms\random_pred.py -> build\lib.win-amd64-3.8\surprise\prediction_algorithms
  copying surprise\prediction_algorithms\__init__.py -> build\lib.win-amd64-3.8\surprise\prediction_algorithms
  running egg_info
  writing scikit_surprise.egg-info\PKG-I

## Importing Libraries

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

import warnings; warnings.simplefilter('ignore')

    writing scikit_surprise.egg-info\PKG-INFO
    writing dependency_links to scikit_surprise.egg-info\dependency_links.txt
    writing entry points to scikit_surprise.egg-info\entry_points.txt
    writing requirements to scikit_surprise.egg-info\requires.txt
    writing top-level names to scikit_surprise.egg-info\top_level.txt
    reading manifest file 'scikit_surprise.egg-info\SOURCES.txt'
    reading manifest template 'MANIFEST.in'
    writing manifest file 'scikit_surprise.egg-info\SOURCES.txt'
    copying surprise\similarities.c -> build\lib.win-amd64-3.8\surprise
    copying surprise\similarities.pyx -> build\lib.win-amd64-3.8\surprise
    copying surprise\prediction_algorithms\co_clustering.c -> build\lib.win-amd64-3.8\surprise\prediction_algorithms
    copying surprise\prediction_algorithms\matrix_factorization.c -> build\lib.win-amd64-3.8\surprise\prediction_algorithms
    copying surprise\prediction_algorithms\optimize_baselines.c -> build\lib.win-amd64-3.8\surprise\predictio

ModuleNotFoundError: No module named 'surprise'

In [ ]:
anime_info_df = pd.read_csv('datset/anime.csv')
anime_desc_df = pd.read_csv('dataset/anime_with_synopsis.csv')
rating_df = pd.read_csv('dataset/rating.csv')

In [ ]:
anime_info_df.head(5)

In [ ]:
anime_df = pd.merge(anime_desc_df,anime_info_df[['MAL_ID','Type','Popularity','Members','Favorites']],on='MAL_ID')
anime_df.head(10)

In [ ]:
anime_df.info()

In [ ]:
anime_df["Score"].describe()

In [ ]:
anime_df = anime_df[(anime_df["Score"] != "Unknown")] 
anime_df.shape

## Content  Filtering

In [ ]:
anime_df['sypnopsis'] = anime_df['sypnopsis'].fillna('')

In [ ]:
tfidf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(anime_df['sypnopsis'])
tfidf_matrix.shape

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

In [ ]:
anime_df = anime_df.reset_index()
titles = anime_df['Name']
indices = pd.Series(anime_df.index, index=anime_df['Name'])

In [ ]:
def content_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    anime_indices = [i[0] for i in sim_scores]
    
    anime_lst = anime_df.iloc[anime_indices][['Name', 'Members', 'Score']]
    favorite_count = anime_lst[anime_lst['Members'].notnull()]['Members'].astype('int')
    score_avg = anime_lst[anime_lst['Score'].notnull()]['Score'].astype('float')
    C = score_avg.mean()
    m = favorite_count.quantile(0.60)
    qualified = anime_lst[(anime_lst['Members'] >= m) & (anime_lst['Members'].notnull()) & (anime_lst['Score'].notnull())]
    qualified['Members'] = qualified['Members'].astype('int')
    qualified['Score'] = qualified['Score'].astype('float')
    def weighted_rating(x):
        v = x['Members']
        R = x['Score']
        return (v/(v+m) * R) + (m/(m+v) * C)   
    
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    
    return qualified

In [ ]:
content_recommendations('Naruto').head(10)

## Collaborative Filtering

In [ ]:
rating_df.head(10)

In [ ]:
rating_df['rating'].value_counts()

In [ ]:
# rating_df['rating'] = rating_df['rating'].apply(lambda x: 1 if x < 0 else x)
rating_df = rating_df[(rating_df["rating"] != -1)] 

In [ ]:
rating_df.head(5)

In [ ]:
reader = Reader()
rating_data = Dataset.load_from_df(rating_df, reader)
svd = SVD()

In [ ]:
trainset = rating_data.build_full_trainset()

In [ ]:
svd.fit(trainset)

In [ ]:
svd.predict(1, 356, 5)

## Hybrid Filtering

In [ ]:
id_map = anime_df[['MAL_ID']]
id_map['id'] = list(range(1,anime_df.shape[0]+1,1))
id_map = id_map.merge(anime_df[['MAL_ID', 'Name']], on='MAL_ID').set_index('Name')

In [ ]:
indices_map = id_map.set_index('id')

In [ ]:
def hybrid_recommendations(user_id,title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]    
    anime_indices = [i[0] for i in sim_scores]
            
    anime_lst = anime_df.iloc[anime_indices][['MAL_ID','Name', 'Members', 'Score','Genres']]
    favorite_count = anime_lst[anime_lst['Members'].notnull()]['Members'].astype('int')
    score_avg = anime_lst[anime_lst['Score'].notnull()]['Score'].astype('float')
    C = score_avg.mean()
    m = favorite_count.quantile(0.60)
    qualified = anime_lst[(anime_lst['Members'] >= m) & (anime_lst['Members'].notnull()) & (anime_lst['Score'].notnull())]    
    qualified['Members'] = qualified['Members'].astype('int')
    qualified['Score'] = qualified['Score'].astype('float')
    def weighted_rating(x):
        v = x['Members']
        R = x['Score']
        return (v/(v+m) * R) + (m/(m+v) * C)   
    
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(30)    
    
    qualified[['id']] = list(range(1,qualified.shape[0]+1,1))  
    qualified['est'] = qualified['id'].apply(lambda x: svd.predict(user_id, indices_map.loc[x]['MAL_ID']).est)
    qualified = qualified.sort_values('est', ascending=False)
    result = qualified[['MAL_ID','Name','Genres','Score']]
    return result.head(10)    

In [ ]:
hybrid_recommendations(8, 'Trigun')